# Übung 2 - Datenvorbereitung und -aggregation
Importiert die Pakete.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

Lesen Sie die Daten "02_Personaldaten.csv" ein und lassen Sie sich die Daten anzeigen.
Damit die Formate richtig importiert werden muss die ANSI-Kodierung mit einem Semikolon als Trenner eingestellt werden.
Da die Personalnummer für die Bezeichnung eines laufenden Indexes etws lang ist, ist sie durch die ID abgekürzt.

In [2]:
df_orig = pd.read_csv("covid_de.csv")
df_orig.index.names = ['ID']



In [3]:
df_orig.head(13)

,state,county,age_group,gender,date,cases,deaths,recovered
ID,,,,,,,,
0,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-03-27,1,0,1
1,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-03-28,1,0,1
2,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-04-03,1,0,1
3,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-10-18,1,0,1
4,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-10-22,1,0,1
5,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-10-27,1,0,0
6,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-10-30,3,0,3
7,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-11-03,1,0,1
8,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-11-07,1,0,1


## 2.1 Datenbereinigung

**Was fällt bei den Daten auf?**

Es sind Lücken und nicht einheitliche Ausprägungen einzelner Attribute zu sehen.

Folgende passende Bereinigungsschritte sind durchzuführen:
1. Attribut Wohnort löschen
2. Geschlecht vereinheitlichen
3. Jahresgehalt in T€ vereinheitlichen
4. Datensatz 12 löschen
5. Betriebszugehörigkeit auf Jahre vereinheitlichen
6. Bildungsabschluss durch häufigsten Wert ersetzen

Führen Sie die Schritte aus und lassen Sie sich das bereinigte Ergebnis anzeigen.

In [153]:
df = df_orig.copy()
# 1. Attribut Wohnort löschen
df.drop(columns=['Wohnort'], inplace=True) #Alternative: df = df.drop(columns=['Wohnort'])

# 2. Geschlecht vereinheitlichen
df.replace({'male':'m', 'fem':'w'}, inplace=True)

# 3. Jahresgehalt in T€ vereinhehitlichen
df.loc[2, ['Jahresgehalt']] = df.loc[2]['Jahresgehalt'] / 1000

# 4. Datensatz 12 löschen
df.drop(12, inplace=True)

# 5. Betriebszugehörigkeit auf Jahre vereinheitlichen
df.loc[6, ['Betriebszugehörigkeit']] = df.loc[6]['Betriebszugehörigkeit'] / 12

# 6. Bildungsabschluss durch häufigsten Wert ersetzen
df['Bildungsabschluss'] = df['Bildungsabschluss'].fillna(df['Bildungsabschluss'].mode()[0])

In [154]:
df.head(13)

,Geschlecht,Alter,Jahresgehalt,Betriebszugehörigkeit,Position,Bildungsabschluss
ID,,,,,,
1,m,45,32.0,10.0,arb,Lehre
2,w,57,35.0,25.0,verw,Bachelor
3,m,52,40.0,5.0,manager,Master
4,m,28,27.0,6.0,arb,Lehre
5,m,57,45.0,25.0,manager,Master
6,w,26,27.0,8.0,arb,Lehre
7,m,39,39.0,4.0,manager,Master
8,m,38,32.0,3.0,arb,Lehre
9,m,42,31.0,15.0,arb,ohne


## 2.2 Datentransformation 1: Diskretisierung
Nun sollen alle numerischen Werte in Wertebereiche (engl. bins) zusammengefasst werden, so dass keine numerischen Daten mehr vorhanden sind. Legen Sie dazu zunächst eine Kopie "dfBins" des bereinigten Dataframes an. 
Neue Ausprägungen: 

- Alter in (jung, alt)
- Jahresgehalt in (wenig, viel)
- Betriebszugehörigkeit in (kurz, lang)

Die Umwandlung von Alter ist bereits vorhanden. Führen Sie die anderen analog aus und lassen sich das Ergebnis anzeigen.


In [155]:
dfBins = df.copy()
bin_labels_alt = ['jung', 'alt']
dfBins['Alter'] = pd.cut(x = df['Alter'],
                              bins=[0, 40, 100],
                              labels = bin_labels_alt)

bin_labels_jahresgehalt = ['wenig', 'viel']
dfBins['Jahresgehalt'] = pd.cut(x = df['Jahresgehalt'],
                              bins=[0, 37, 100],
                              labels = bin_labels_jahresgehalt)

bin_labels_betriebszugehoerigkeit = ['kurz', 'lang']
dfBins['Betriebszugehörigkeit'] = pd.cut(x = df['Betriebszugehörigkeit'],
                              bins=[0, 12, 100],
                              labels = bin_labels_betriebszugehoerigkeit)#
dfBins

,Geschlecht,Alter,Jahresgehalt,Betriebszugehörigkeit,Position,Bildungsabschluss
ID,,,,,,
1,m,alt,wenig,kurz,arb,Lehre
2,w,alt,wenig,lang,verw,Bachelor
3,m,alt,viel,kurz,manager,Master
4,m,jung,wenig,kurz,arb,Lehre
5,m,alt,viel,lang,manager,Master
6,w,jung,wenig,kurz,arb,Lehre
7,m,jung,viel,kurz,manager,Master
8,m,jung,wenig,kurz,arb,Lehre
9,m,alt,wenig,lang,arb,ohne


## 2.3 Datentransformation 2: Numerische Kodierung und Normierung
Nun sollen alle Daten in normierte numerische Werte transformiert werden. Legen Sie auch hierfür eine Kopie "dfNumeric" des originären bereinigten Dataframes an.
1. Ausprägungen von Geschlecht, Position und Bildung durch numerische aufeinanderfolgende Werte (1,2,...) ersetzen. 
2. Werte auf das Intervall [0, 1] normalisieren. Hierfür wird der MinMaxScaler() verwendet, den Sie oben importiert haben.

Lassen Sie sich erneut das Ergebnis ausgeben.

In [162]:
dfNumeric  = df.copy()
dfNumeric['Geschlecht'].replace(to_replace=['m', 'w'], value=[1, 2], inplace=True)
dfNumeric['Position'].replace(to_replace=['arb', 'verw', 'manager'], value=[1, 2, 3], inplace=True)
dfNumeric['Bildungsabschluss'].replace(to_replace=['Lehre', 'Bachelor', 'Master', 'ohne', 'Abi'], value=[1, 2, 3, 4 ,5], inplace=True)
dfNumeric.head()

,Geschlecht,Alter,Jahresgehalt,Betriebszugehörigkeit,Position,Bildungsabschluss
ID,,,,,,
1,1,45,32.0,10.0,1,1
2,2,57,35.0,25.0,2,2
3,1,52,40.0,5.0,3,3
4,1,28,27.0,6.0,1,1
5,1,57,45.0,25.0,3,3


In [163]:
scaler = MinMaxScaler()

dfNumeric = pd.DataFrame(scaler.fit_transform(dfNumeric), columns = dfNumeric.columns, index = dfNumeric.index)


Überprüfen Sie mit einem geeigneten Aufruf, ob die Skalierung erfolgreich war.

In [166]:
dfNumeric.describe().drop(['count', 'mean', 'std', '25%', '50%', '75%'])

,Geschlecht,Alter,Jahresgehalt,Betriebszugehörigkeit,Position,Bildungsabschluss
min,0.0,0.0,0.0,0.0,0.0,0.0
max,1.0,1.0,1.0,1.0,1.0,1.0


## 2.4 Datentransformation 3: Binärkodierung der Kategorien
Nun soll auf den bereinigten Daten eine Binärkodierung (One-to-Many) der kategorischen Variablen durchgeführt werden. Dafür verwenden wir eine Kopie "dfBinaer". Normieren Sie das Ergebnis wie in Schritt 2.3.


In [167]:
dfBinaer = df.copy()
dfBinaer = pd.get_dummies(dfBinaer, prefix = '', prefix_sep = '')
dfBinaer

,Alter,Jahresgehalt,Betriebszugehörigkeit,m,w,arb,manager,verw,Abi,Bachelor,Lehre,Master,ohne
ID,,,,,,,,,,,,,
1,45,32.0,10.0,1,0,1,0,0,0,0,1,0,0
2,57,35.0,25.0,0,1,0,0,1,0,1,0,0,0
3,52,40.0,5.0,1,0,0,1,0,0,0,0,1,0
4,28,27.0,6.0,1,0,1,0,0,0,0,1,0,0
5,57,45.0,25.0,1,0,0,1,0,0,0,0,1,0
6,26,27.0,8.0,0,1,1,0,0,0,0,1,0,0
7,39,39.0,4.0,1,0,0,1,0,0,0,0,1,0
8,38,32.0,3.0,1,0,1,0,0,0,0,1,0,0
9,42,31.0,15.0,1,0,1,0,0,0,0,0,0,1


In [168]:
scaler = MinMaxScaler()

dfNumericBinaer = pd.DataFrame(scaler.fit_transform(dfBinaer), columns = dfBinaer.columns, index = dfBinaer.index)

Welche der beiden Varianten der Kodierung halten Sie in dem Beispiel für geeigneter?

In [169]:
dfNumericBinaer

,Alter,Jahresgehalt,Betriebszugehörigkeit,m,w,arb,manager,verw,Abi,Bachelor,Lehre,Master,ohne
ID,,,,,,,,,,,,,
1,0.612903,0.277778,0.318182,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.000000,0.444444,1.000000,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.838710,0.722222,0.090909,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.064516,0.000000,0.136364,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1.000000,1.000000,1.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.000000,0.000000,0.227273,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,0.419355,0.666667,0.045455,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.387097,0.277778,0.000000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0.516129,0.222222,0.545455,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
